# Overview

In this project, I am a Data Scientist contractor working with different Women's Fashion Brands, such as Zara and H&M, to classify comments as negative or positive. This project aims to create a generalizable classification model that can be fed comments from any platform, such as Twitter, Instagram, or blogs. For demonstration purposes, I will utilize my model to classify tweets as positive or negative so the customer service teams can respond appropriately, such as responding to customer concerns in negative tweets and retweeting positive tweets.

# Business Understanding

Natural language processing (NLP) has proved to be a highly effective technology for companies to save time and money while optimizing business processes. One of the major benefits of natural language processing for businesses is the ability to process massive volumes of text across various outlets, such as social media platforms, blogs, comments, and so on. By utilizing a machine learning model to classify text as positive or negative, businesses can send the comments to the appropriate teams, such as Quality Assurance for negative comments or Marketing for positive comments. Social media is the second most preferred channel (after an in-person interaction) for customers to initiate communication; for millennials, social media is the most preferred channel. It is important for businesses to respond to these comments in order to protect the brand's reputation, show customers your care, reduce churn, and improve visibility to potential new customer bases and improve customer loyalty for current customers. Generalizability is important with the constant evolution of technology, social media, and how users choose to interact with products and brands. While one brand may receive lots of tweets, another brand may be discussed more in the comments of their Instagram posts.

## Use Cases:

Positive Comments
- Retweet or repost from the Brand account
- Respond to boost that comment and reach more viewers
- Identify positive trends, such as colors or fit types, that multiple users are responding to
- Improve brand loyalty

Negative Comments
- Respond to help customers feel heard and solve any fixable issues
- Identify issues with items before next drops, such as colors not photographing accurately on website or quality issues with fabric
- Improve brand loyalty 

## Why utilize data from reviews?

Reviews left by customers contain both text reviews (the meat of the review) in conjunction with a rating. By utilizing reviews as my source, I am able to easily assign a label of Positive_Rating (0 being false, 1 being true) to the review text based on the rating. This helps my model learn what words are associated with a positive sentiment and what words may be more associated with a negative sentiment. I can then apply this model to texts that are not accompanied by a number rating, such as tweets, Instagram captions, blog posts, and more.

# Data

Data for this project was pulled from a compiled dataset of Women's E-Commerce Clothing Reviews compiled in one CSV file. 

The review data contained 23,486 reviews. Additional variables included Clothing ID, Age, Title (of the review if there was one), Review Text, Rating, Recommended IND (whether or not the customer recommends a product), Positive Feedback Count (number of other customers who found the review positive), Division Name (categorical name of product high-level division), Department Name, and Class Name. Reviews ranged on a scale of 1-5. A majority of reviews received an overall rating of 5, which could be a limitation to the model.

### Loading Necessary Imports

In [1]:
#Basic imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
%matplotlib inline

from sklearn.model_selection import train_test_split

#Required text pre-processing libraries are imported
import string
import nltk
import re

# download the stopwords and wordnet corpus
nltk.download('stopwords')
from nltk.corpus import stopwords
# import tokenize from nltk library
from nltk import tokenize
# import WordNetLemmatizer from nltk library
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import CountVectorizer
from nltk.probability import FreqDist
from nltk.tokenize import RegexpTokenizer

#Modeling imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jillian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
### Loading in Tweets

In order to obtain new data, I utilized Twitter's API and made API calls to obtain tweets that were related to women's clothing items. Specifically, I made queries directed @ZARA and @h&m that included words such as "dress," "shirt," and "clothing." I then ran these tweets through a preprocessing function so they could be run through my model. My model is able to return a classification for each tweet along with probability of each class (0 or 1).



In [3]:
tweets=pd.read_csv('./data/final_tweet_dataset.csv')

In [4]:
tweets

,id,created_at,text,lang,conversation_id,reply_settings,referenced_tweets,in_reply_to_user_id,source,author_id,public_metrics,geo
0,1560364876853280770,2022-08-18T20:35:51.000Z,"@EJGSH @ZARA I tried, literally nobody gives a...",en,1560344787659661312,everyone,"[{'type': 'replied_to', 'id': '156035492260387...",3.001121e+07,Twitter for iPhone,23023560,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN
1,1560349998528724995,2022-08-18T19:36:44.000Z,Are @ZARA kids clothes true to size? or a litt...,en,1560349998528724995,everyone,NaN,NaN,Twitter for iPhone,1162452050896203776,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN
2,1560344787659661312,2022-08-18T19:16:01.000Z,Bought a pair of pants from @ZARA had them for...,en,1560344787659661312,everyone,NaN,NaN,Twitter for iPhone,23023560,"{'retweet_count': 0, 'reply_count': 2, 'like_c...",NaN
3,1560263688048689152,2022-08-18T13:53:46.000Z,@climatecorporat @ZARA @UniqloUSA @hm Anything...,en,1560238655033053184,everyone,"[{'type': 'replied_to', 'id': '156024155013919...",1.403416e+18,Twitter for Android,2837589067,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN
4,1560035225190776846,2022-08-17T22:45:56.000Z,Disappointing service from @ZARA. \nOrdered a...,en,1560035225190776846,mentionedUsers,NaN,NaN,Twitter for Android,1433752245242408964,"{'retweet_count': 0, 'reply_count': 1, 'like_c...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
74,1558452009459748868,2022-08-13T13:54:48.000Z,@greenwashdotcom @_DirtyFashion_ @qz @hm @Higg...,en,1558435691989975041,everyone,"[{'type': 'replied_to', 'id': '155843569198997...",1.491801e+18,Twitter for iPhone,1224824519996723206,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN
75,1558364150518239232,2022-08-13T08:05:41.000Z,@avneetkaur_13 I wore this outfit yesterday bu...,en,1558364150518239232,everyone,NaN,7.880625e+17,Instagram,1437114287894327298,"{'retweet_count': 0, 'reply_count': 0, 'like_c...","{'place_id': '7929cea6bd5b32bd', 'coordinates'..."
76,1558359664215179264,2022-08-13T07:47:51.000Z,#sjcartierltd #reselling #brandnew &amp; #used...,en,1558359664215179264,everyone,NaN,NaN,Twitter for Android,1273298286204661760,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN
77,1558302374686588928,2022-08-13T04:00:12.000Z,i liked my outfit yesterday 🙆🏻‍♀️\npolo: @uniq...,en,1558302374686588928,everyone,NaN,NaN,Instagram,95186322,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",NaN


## Basic Generalizable Preprocessing

In [5]:
#need general preprocessing function to clean my tweets--lowercase, remove punctuation, remove emojis, etc.

In [6]:
#utilizing same function I used for review data

In [7]:

# Create a tokenizer 
pattern = "([a-zA-Z]+(?:'[a-z]+)?)"
tokenizer = RegexpTokenizer(pattern)

# Create list of stopwords in English (language of the reviews)
stopwords_list = stopwords.words("english")
# Remove "very" from the list of stopwords 
stopwords_list.remove("very")

# Create an instance of nltk's WordNetLemmatizer with the variable name `lemmatizer`
lemmatizer = WordNetLemmatizer()

In [8]:
def preprocess_text(text, tokenizer, stopwords_list, lemmatizer):
    # Standardize case (lowercase the text)
    lowered_text = text.lower()
    
    # Tokenize text using `tokenizer`
    tokens = tokenizer.tokenize(lowered_text)
    
    # Remove stopwords using `stopwords_list` and removing punctuation and strings with non-alphabetic properties 
    stopped_tokens = [word for word in tokens if word not in stopwords_list and word not in string.punctuation and word.isalpha()]
    
    # Lemmatize the tokenized text using `lemmatizer`
    lemmatized_text = [lemmatizer.lemmatize(token) for token in stopped_tokens]
    
    # Return the preprocessed text
    return lemmatized_text

In [9]:
text = tweets['text']

In [10]:
text

0     @EJGSH @ZARA I tried, literally nobody gives a...
1     Are @ZARA kids clothes true to size? or a litt...
2     Bought a pair of pants from @ZARA had them for...
3     @climatecorporat @ZARA @UniqloUSA @hm Anything...
4     Disappointing service from @ZARA.  \nOrdered a...
                            ...                        
74    @greenwashdotcom @_DirtyFashion_ @qz @hm @Higg...
75    @avneetkaur_13 I wore this outfit yesterday bu...
76    #sjcartierltd #reselling #brandnew &amp; #used...
77    i liked my outfit yesterday 🙆🏻‍♀️\npolo: @uniq...
78    #outfitoftheday @macys jewelry @CalvinKlein he...
Name: text, Length: 79, dtype: object

In [11]:
text = text.apply(lambda x: preprocess_text(x, tokenizer, stopwords_list, lemmatizer))

In [12]:
text

0     [ejgsh, zara, tried, literally, nobody, give, ...
1     [zara, kid, clothes, true, size, little, smaller]
2     [bought, pair, pant, zara, two, week, hole, ri...
3     [climatecorporat, zara, uniqlousa, hm, anythin...
4     [disappointing, service, zara, ordered, pair, ...
                            ...                        
74    [greenwashdotcom, dirtyfashion, qz, hm, higgda...
75    [avneetkaur, wore, outfit, yesterday, going, d...
76    [sjcartierltd, reselling, brandnew, amp, used,...
77    [liked, outfit, yesterday, polo, uniqlophoffic...
78    [outfitoftheday, macys, jewelry, calvinklein, ...
Name: text, Length: 79, dtype: object

In [13]:
text = text.str.join(" ")

In [14]:
text

0     ejgsh zara tried literally nobody give shit pi...
1             zara kid clothes true size little smaller
2     bought pair pant zara two week hole ripped sea...
3     climatecorporat zara uniqlousa hm anything tha...
4     disappointing service zara ordered pair trouse...
                            ...                        
74    greenwashdotcom dirtyfashion qz hm higgdata h ...
75    avneetkaur wore outfit yesterday going differe...
76    sjcartierltd reselling brandnew amp used colle...
77    liked outfit yesterday polo uniqlophofficial d...
78    outfitoftheday macys jewelry calvinklein heel ...
Name: text, Length: 79, dtype: object

## Instantiating and Fitting Final Model to Create Predictions

In [15]:
df = pd.read_csv('./data/final_data.csv')

In [16]:
# Specify X as the cleaned strings in df and y as the target-Positive_Rating.
X = df['String']
y = df['Positive_Rating']

In [17]:
#Performed the train-test split, using 20% for the hold-out data.
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    random_state=42,
                                                    test_size=0.20)

In [18]:
final_model = Pipeline([
           ('vect', TfidfVectorizer(max_df=.35, max_features=3000, ngram_range=(1,2))),
           ('lr', LogisticRegression(random_state=42, solver='lbfgs', C=3, max_iter=30))])

In [19]:
final_model.fit(X_train, y_train)

/Users/jillian/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('vect',
                 TfidfVectorizer(max_df=0.35, max_features=3000,
                                 ngram_range=(1, 2))),
                ('lr', LogisticRegression(C=3, max_iter=30, random_state=42))])

In [20]:
final_model.score(X_train, y_train)

0.9122128975265018

In [21]:
#confirming same score; fit was correct--will run tweets through model

In [22]:
predictions = final_model.predict(text)

In [23]:
text

0     ejgsh zara tried literally nobody give shit pi...
1             zara kid clothes true size little smaller
2     bought pair pant zara two week hole ripped sea...
3     climatecorporat zara uniqlousa hm anything tha...
4     disappointing service zara ordered pair trouse...
                            ...                        
74    greenwashdotcom dirtyfashion qz hm higgdata h ...
75    avneetkaur wore outfit yesterday going differe...
76    sjcartierltd reselling brandnew amp used colle...
77    liked outfit yesterday polo uniqlophofficial d...
78    outfitoftheday macys jewelry calvinklein heel ...
Name: text, Length: 79, dtype: object

In [24]:
predictions

array([1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1])

In [25]:
len(predictions)

79

In [26]:
text_df = pd.DataFrame(tweets['text'])

In [27]:
text_df

,text
0,"@EJGSH @ZARA I tried, literally nobody gives a..."
1,Are @ZARA kids clothes true to size? or a litt...
2,Bought a pair of pants from @ZARA had them for...
3,@climatecorporat @ZARA @UniqloUSA @hm Anything...
4,Disappointing service from @ZARA. \nOrdered a...
...,...
74,@greenwashdotcom @_DirtyFashion_ @qz @hm @Higg...
75,@avneetkaur_13 I wore this outfit yesterday bu...
76,#sjcartierltd #reselling #brandnew &amp; #used...
77,i liked my outfit yesterday 🙆🏻‍♀️\npolo: @uniq...


In [28]:
predictions_df = pd.DataFrame(predictions)

In [29]:
predictions_df

,0
0,1
1,1
2,0
3,1
4,0
...,...
74,1
75,1
76,1
77,1


In [30]:
#combining original tweets with predicted classification for visual inspection

In [31]:
classification_df = text_df.join(predictions_df)

In [32]:
classification_df

,text,0
0,"@EJGSH @ZARA I tried, literally nobody gives a...",1
1,Are @ZARA kids clothes true to size? or a litt...,1
2,Bought a pair of pants from @ZARA had them for...,0
3,@climatecorporat @ZARA @UniqloUSA @hm Anything...,1
4,Disappointing service from @ZARA. \nOrdered a...,0
...,...,...
74,@greenwashdotcom @_DirtyFashion_ @qz @hm @Higg...,1
75,@avneetkaur_13 I wore this outfit yesterday bu...,1
76,#sjcartierltd #reselling #brandnew &amp; #used...,1
77,i liked my outfit yesterday 🙆🏻‍♀️\npolo: @uniq...,1


In [33]:
pd.set_option('display.max_colwidth', None)

In [34]:
classification_df.rename(columns={'text': 'Tweet', 0: "Predicted_Class"}, inplace=True)

In [35]:
classification_df[:10]

,Tweet,Predicted_Class
0,"@EJGSH @ZARA I tried, literally nobody gives a shit. I’m so pissed $40 pants and they’re just like “sorry buy a new pair”",1
1,Are @ZARA kids clothes true to size? or a little smaller?,1
2,Bought a pair of pants from @ZARA had them for two weeks before a hole ripped in the seam. I didn’t have my receipt and they told my there was nothing they could do. Greedy fucks with shit quality clothing.,0
3,"@climatecorporat @ZARA @UniqloUSA @hm Anything. Thats produced for mass individual consumption cannot be sustainable . Bottled water, cars, packaged food, Clothes , shoes , Mobile , TV , Tablets . Everything adds to Waste dump. Faster than ever before.",1
4,"Disappointing service from @ZARA. \nOrdered a pair of trousers online and a pair of cargo pants showed up at my door 😱. Same barcode, so clearly issues at warehousing process. \nPlease sort ASAP. https://t.co/vMe4JlT5Hf",0
5,"@ZARA I bought a dark khaki bodysuit at Zara. I followed the instructions to wash it, but the colour of my bodysuit came out and it stained to my other clothes. My bodysuit is dark in someplace and light in someplace. The main thing is I haven’t even worn it. https://t.co/eLA88o2OoK",0
6,@UnderRynovation @ZARA Girl they gave me the whole run around talking about I couldn’t return a shirt that was damaged when they sold it to me and tried to say it was me,1
7,"@ZARA Zara, I'm a huge fan of your clothes, as are so many of my friends, none of us are a size 6 or 8, please add some variation in your models so we can get a better idea what clothes look like on our size bodies",0
8,@CaroleGillis @Lammles @Wrangler Ooof I had a fight with Zara over something similar. Finally told them forget it and my mom fixed the hole. Customer service was abysmal and @ZARA lost a customer for life over a pair of $50 jeans. Hope it was worth it for them cuz my kid sure likes to spend money on clothes😂,0
9,@zara fuck you for not letting buy my skirt,1


In [36]:
#Incorrectly classed tweets:
#tweet 0, 3, 6, 7, 9 is clearly negative or need improvements and predicted as positive

In [37]:
#Correctly classed tweets:
#tweet 2, 4, 5, 8

In [38]:
#tweet 1 is neutral

In [39]:
classification_df[10:20]

,Tweet,Predicted_Class
10,Queen B said it best 💅🏽✨ Dress is from @zara Heels @express Watch is @mvmtforher #grwmreel #fashionstyle #zaradress #pinkoutfit #goingoutoutfit #datenightoutfit https://t.co/t0klk5Ofyi,1
11,"💚\n\nJeans: @zara \nShoe: @zara \nShirt: @degeorgescouture \nWatch: @casiovintage_eu \n\n#slimchoko #chokovibes #fashiontrend #summeroutfit #modelsearch #zaraoutfit #zaramen #zarachocolate @ Northampton, Northamptonshire https://t.co/mYhtD4xCxd",1
12,Bella Vita 🌺\n———\nTop: @freepeople \nPants: @targetstyle \nSandals: @zara \nSunnies: @amazonfashion \nMood: chill getaway.\nPhotos: @ruzz 🌺\n#makeupbyjesi https://t.co/TVKGPCmmXM,1
13,"breezy summer vibe 🕊🧺\nParkdale: @ZARA cot/poly/visc dress size S $38, sunnies $15, woven handbag $24 and Rockey Dog cork wedges size 10 $24 #thriftshop https://t.co/GwHqxJH9oR",1
14,"@ZARA @ZARA_Care Have purchased clothes worth INR 50,000 from you &amp; your store is not ready to accept returns. You say 30 day return policy but that is a big scam. Your customer support is helpless and rude. Very disappointed with this experience. ORDER Id: 52950094142\n#pathetic",0
15,@ZARA can I get free clothes for finding your website’s typos #lenght https://t.co/roDRNz0jql,1
16,"@TheSocialCTV @ZARA @Simone_Rocha_ @CHANEL @The_Kooples @aliceandolivia @SteveMadden @31philliplim @GreymerOfficial Fall colours already why the rush.Cute Melissa surprise no mini skirt,lol",1
17,@ZARA I ordered the 1st dress (only offered in paisley) and received the 2nd. Customer support refused to waive the return fee for a dress I did not order and told me that I “received the correct dress.” 🤯 Accusing your customers of lying isn’t a good look - please fix this. https://t.co/sJjM4PxuwG,1
18,.@Zara. The second thing that maybe you shouldn’t do is call over another colleague who doubles down and tells the kid clutching a pair of jeans that they need to go to the mens fitting room if trying on mens clothes (the jeans were from your womens department fyi) @ZARA_Care,1
19,This is us kicking off the last week of Season 9!!!! 🥳\n\nMel\nOutfit: @ZARA \n\nLainey \nDress: @Simone_Rocha_\nShoes: @CHANEL \n\nCynthia \nTop: @The_Kooples\nPants: @aliceandolivia\nShoes: @SteveMadden\n\nAndrea\nDress: @31philliplim\nShoes: @GreymerOfficial\n\n#WhatWeWore #OOTD https://t.co/ElQ4xX0nUT,1


In [40]:
#correctly classed tweets:
#tweets 10, 11, 12, 13, 16, 19 all seem to be rewteetable OOTDs
#tweet 14 correctly classed as negative

In [41]:
#Incorrectly classed tweets:
#tweet 17, 18

In [42]:
#netural tweets:
#15

In [43]:
classification_df[20:40]

,Tweet,Predicted_Class
20,@ZARA ilsym for my kids clothes but pls pls pls build a functioning website.,0
21,"1/\nWeb3 Fashion by day 💃🏻 and NFT Fashion by night 🦸🏻‍♀️\n\n💃🏻 Day \n@MAVIONworld necklace\n@ZARA top, skirt\n@nike dunks, bag\n\n🦸🏻‍♀️ Night\n@ZARA Top, Blazer, Pants\n@Nike dunks\nBottega Bag\n\nWhich one do you like better?\n💃Day or 🦸‍♀️Night?\n\n#fashion #web3fashion #fitcheck https://t.co/3qb6ML7wJ2",1
22,"@ZARA I deserve a discount code or something. A free return label, and a refund isn’t enough. Now I have to wait 3-7 business days for that money back, and now I’m out a dress for my vacation. Without any real time to find a replacement. https://t.co/S16FERKGck",1
23,Ordered a dress for my vacation from @zara … tell me why this is what they sent me. https://t.co/pCEldoqXxV,1
24,"Look at shirt in @ZARA when @goodmindsnft TV’s and breads, et cetera on these shirts??? Make these connects! Your art is Superior 💜 https://t.co/B8dAqVHbrb",1
25,@ZARA shirt arriving creased to hell is one thing but burn marks on front and back? Immediate return #badquality https://t.co/xPYXDyd3wq,0
26,"Let your spirit Fly with Peace, Happiness and Positivity\nGreen is so cool, so zen and yet makes such a statement.\nShirt and Footwear @zara Pants @marksandspencerindia\n✨ #greenbeautyblogger #sass #sasswars #attitudegirls #greenfashion #wearinggreen #attitudeiseverything #AJ https://t.co/s2dn5SxcPi",1
27,We’ve just had the most amazing conversation on The G&amp;T Show about @eddieizzard and THAT @ZARA DRESS!!!!! Me and @LGBwiththeT love you Eddie and you looked FIERCE!!!! Listen in on @transradiouk,1
28,"Hey @eddieizzard we LOVE you! Please come chat to me and @LGBwiththeT and tell us how fabulous you are….,.listen in now as we’re talking about your @ZARA dress ❤️❤️ https://t.co/EqEO5PcgAj",1
29,We’re about to discuss the fact @eddieizzard wears THAT @ZARA dress better than anyone else and looks FIERCE!!!! Me &amp; @LGBwiththeT both love you Eddie and we wish you’d come on our show! Tune in now on @transradiouk,1


## Analysis of my Model's Performance

From the inspection of my model, it is clear that tweets that are predicted as 0 (negative) are most likely correct and need to be immediately addressed. Tweets that are addressed as 1 are more likely to possibly be class 0 and will require further inspection by a human. This is most likely due to the class imbalance of my inital dataset and my model having more examples of positive reviews than negative reviews.

## Saving the Trained Model

In [44]:
import pickle

In [45]:
filename='trained_model.sav'
pickle.dump(final_model, open(filename, 'wb'))

# Limitations


Some limitations effecting my model include:
- Class Imbalance: almost 80% of the reviews were coded as "positive" based on a rating of 4 or 5
- User error when rating items: based on some of the reviews, it appears as though users may have mixed up the ratings and utilized 1 for positive reviews or 5 for negative reviews
    

# Next Steps

Given more time, I would expand this product by:
- Pulling in more data, specifically negative reviews or comments to help better train my model
- Create a dual classification system that first classifys comments as Spam or Not Spam, and then filters out Spam comments and classifies Not Spam as Positive or Negative
- Test my model with other platforms, such as Instagram comments
- Creating a "neutral" target for classification
- Create custom stopwords list, such as removing "not" from stopwords (may be helpful for my model when looking at bigrams)
- Upgrade and deploy an app for brands to utilize to perform Twitter API calls based on user inputted query terms and return a dataframe with one column of tweets and one column of classification

# Conclusion

In conclusion, using my generalizable model will allow women's fashion brand companies to feed comments from any web source to identify positively and negatively sentiment comments and respond appropriately. Utilizing the model will allow companies to save time and by flagging the comments for appropriate teams to respond to, such as Quality Assurance to respond to negative comments, or marketing to respond to and promote positive comments. Responding to these comments appropriately will improve products by identifying potential clothing quality issues if the same sentiment is being repeated, identify potential positive trends, boost visability by responding to comments, and improve brand loyalty by making customers feel heard.